In [1]:
import simtk
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
import numpy as np
from simtk import unit
import copy
sys.path.append('../')
import itertools
from sys import stdout


topology.Topology.loadBondDefinitions('./popc.xml')
output_directory = './'
pdb = PDBFile('./step5_assembly.pdb')
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')

water_coords = np.array([pos/nanometer for pos, atom in zip(pdb.positions, pdb.topology.atoms()) if atom.residue.name=='HOH'])
size = water_coords.max(0) - water_coords.min(0)
pdb.topology.setPeriodicBoxVectors([Vec3(x=size[0], y=0, z=0), 
                                   Vec3(x=0, y=size[1], z=0), 
                                   Vec3(x=0, y=0, z=size[2])])

In [2]:
#get the indices of the 'ligand', in this case a sodium ion:
ligand_idx = [i.index for i in pdb.topology.atoms() if i.residue.name == 'NA'][0:1]
ligand_idx

[35529]

In [3]:
#get the indices of the lipid headgroups.
#it's convenient to use the phosphorus but you could choose any atom, or
#group of atoms to create centroid bond force.

#note: the leaflets can be separated by z coordinate > or < zero in this pdb that
#came from CHARMMGUI, but that might change particularly when reloading an OpenMM sim
#that shifts coords to have minimum [0,0,0]

#fetch the indices of a single leaflet:
upper_leaflet_idx = [atom.index for atom, position in zip(pdb.topology.atoms(), pdb.positions) if (position[2]>0*nanometer and atom.name=='P')]

#fetch the other leaflet:
lower_leaflet_idx = [atom.index for atom, position in zip(pdb.topology.atoms(), pdb.positions) if (position[2]<0*nanometer and atom.name=='P')]


In [4]:
###Creating a force controlling distance to upper leaflet headgroup:


energy_expression = "distance1 = distance(g1, g2)"
for group in range(1, len(upper_leaflet_idx)):
    last_term = "distance"+str(group+1)
    energy_expression = last_term + " = min(distance"+str(group)+", distance(g1,g"+str(group+2)+"));"+energy_expression
    
energy_expression = "j*"+last_term+";"+energy_expression

force_upper = CustomCentroidBondForce(1+len(upper_leaflet_idx), energy_expression)
ligand = force_upper.addGroup(ligand_idx)

upper_headgroup_groups = list()
for atom in upper_leaflet_idx:
    upper_headgroup_group = force_upper.addGroup([atom])
    upper_headgroup_groups.append(upper_headgroup_group)

force_upper.addBond([0]+upper_headgroup_groups, [])


#make sure it's periodic!
force_upper.setUsesPeriodicBoundaryConditions(True)

#add a multiplier that is just 0, so the simulation won't actually be affected by the force
force_upper.addGlobalParameter('j', 0*kilojoule/mole)

0

In [5]:
###Creating a force controlling distance to lower leaflet headgroup:

energy_expression = "distance1 = distance(g1, g2)"
for group in range(1, len(upper_leaflet_idx)):
    last_term = "distance"+str(group+1)
    energy_expression = last_term + " = min(distance"+str(group)+", distance(g1,g"+str(group+2)+"));"+energy_expression
    
energy_expression = "k*"+last_term+";"+energy_expression


force_lower = CustomCentroidBondForce(1+len(lower_leaflet_idx), energy_expression)
ligand = force_lower.addGroup(ligand_idx)

lower_headgroup_groups = list()
for atom in lower_leaflet_idx:
    lower_headgroup_group = force_lower.addGroup([atom])
    lower_headgroup_groups.append(lower_headgroup_group)

force_lower.addBond([0]+lower_headgroup_groups, [])

#make sure it's periodic!
force_lower.setUsesPeriodicBoundaryConditions(True)
#add a multiplier that is just 0, so the simulation won't actually be affected by the force
force_lower.addGlobalParameter('k', 0*kilojoule/mole)

0

In [6]:
##Print out the energy expression to see what just happened:

##it works like this: g1 is the ligand group. g2 through g'N+1' are the phosphorus atoms.
##with N=100 lipids, you will have up to group g101 (here there are 66 lipids) 
##The energy expression iterates through distances (g1,g2), (g1,g3), (g1,g4) and keeps a memory
##of the minimum distance encountered so far. By the Nth distance encountered, all distances have been seen
##and the total minimum should be stored in the variable 'distanceN'

energy_expression

'k*distance66;distance66 = min(distance65, distance(g1,g67));distance65 = min(distance64, distance(g1,g66));distance64 = min(distance63, distance(g1,g65));distance63 = min(distance62, distance(g1,g64));distance62 = min(distance61, distance(g1,g63));distance61 = min(distance60, distance(g1,g62));distance60 = min(distance59, distance(g1,g61));distance59 = min(distance58, distance(g1,g60));distance58 = min(distance57, distance(g1,g59));distance57 = min(distance56, distance(g1,g58));distance56 = min(distance55, distance(g1,g57));distance55 = min(distance54, distance(g1,g56));distance54 = min(distance53, distance(g1,g55));distance53 = min(distance52, distance(g1,g54));distance52 = min(distance51, distance(g1,g53));distance51 = min(distance50, distance(g1,g52));distance50 = min(distance49, distance(g1,g51));distance49 = min(distance48, distance(g1,g50));distance48 = min(distance47, distance(g1,g49));distance47 = min(distance46, distance(g1,g48));distance46 = min(distance45, distance(g1,g47))

In [7]:
def make_system(ff, top):
    system = ff.createSystem(top, 
                        nonbondedMethod = PME,
                        nonbondedCutoff = 0.8*nanometer,
                        constraints = HBonds,
                        rigidWater = True, )
    system.addForce(MonteCarloBarostat(1*atmosphere, 310*kelvin))
    return system


def make_simulation(system, top):
    integrator = LangevinIntegrator(310*kelvin, 1/picosecond, 0.002*picoseconds)
    simulation = Simulation(top, system, integrator)
    simulation.context.setPositions(pdb.positions)
    simulation.minimizeEnergy()
    simulation.context.setVelocitiesToTemperature(310*kelvin)
    return simulation

# Record speed without extra force:

In [8]:
system = make_system(forcefield, pdb.topology)
simulation = make_simulation(system, pdb.topology)
#simulation.reporters.append(DCDReporter('equil.dcd', 1000))
simulation.reporters.append(StateDataReporter(stdout, 500, step=True, potentialEnergy=True, temperature=True, speed=True))
simulation.step(10000)

#"Step","Potential Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
500,-415377.830598661,256.0107483721364,0
1000,-404148.72401390853,289.49824048182285,180
1500,-402392.5382541686,304.7050763262747,180
2000,-402428.72185623134,308.06320291874067,179
2500,-401350.9437954135,309.15808706038274,179
3000,-401362.6921109848,307.968768743152,178
3500,-402157.9397244232,310.51282293168356,178
4000,-402799.70373091963,309.47532029901384,178
4500,-404256.47876604204,310.81734866722917,178
5000,-404561.3674866776,310.83925085683205,178
5500,-404088.63524155365,310.31296054434347,178
6000,-404840.8837782014,309.1754830670573,178
6500,-406227.272890107,313.0699386814571,178
7000,-404946.8962762791,308.56106087300964,178
7500,-405913.4937771554,308.8300976888234,178
8000,-405182.3134579973,309.38766896822733,178
8500,-406341.3977689741,311.5614939100795,178
9000,-406870.29470313806,312.37866067182455,178
9500,-406250.25415351056,311.8305976204804,178
10000,-407747.10493959207,315.389281267383

# Record speed with extra force:
It takes a little while to compile the force because of the long energy expression.

In [9]:
system = make_system(forcefield, pdb.topology)
system.addForce(force_upper)
system.addForce(force_lower)
simulation = make_simulation(system, pdb.topology)
#simulation.reporters.append(DCDReporter('equil.dcd', 1000))
simulation.reporters.append(StateDataReporter(stdout, 500, step=True, potentialEnergy=True, temperature=True, speed=True))
simulation.step(10000)

#"Step","Potential Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
500,-412223.72329741437,254.82324350460019,0
1000,-400669.3782048235,290.7939042315442,142
1500,-398098.8217189687,304.650696663174,142
2000,-397043.79509514966,306.8173450148996,142
2500,-397357.7319211252,310.1437078280292,142
3000,-395769.7443494997,309.1655716858406,142
3500,-398491.4489374638,311.35779548322887,142
4000,-398801.9341898635,312.1881316774778,142
4500,-399642.41060008225,311.0975667872707,142
5000,-401948.7248759959,309.3285281147318,142
5500,-402353.72479629633,309.8868026200361,142
6000,-402644.688984307,307.02806557361345,142
6500,-402218.9454325733,308.5166657162999,142
7000,-402931.90686796466,309.70454729260547,142
7500,-403491.2462539759,308.5589728358646,142
8000,-404549.14444976114,310.2051967636063,141
8500,-403921.75729866,309.94427209592277,141
9000,-403897.6860086748,310.8566292846238,141
9500,-404010.46805329504,310.2692299315364,141
10000,-404589.293699238,310.39470815124736,141


In [10]:
141 / 178

0.7921348314606742